In [7]:
#!/usr/bin/env python3

# Global imports
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
import GlobalSets.c2l_functions as c2l

# Import librarys
from pymongo import collection, errors
import bson

def local2cloud():
    while(True):
        for collection in col.Collections:
                c2l.downloadBase(database=db.dbBuffer, collection=collection['name'])

if __name__ == '__main__':
    try:
        local2cloud()
    except Exception:
        pass


In [ ]:
#!/usr/bin/env python3

# Global imports
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col

# Import librarys
from pymongo import collection, errors
from bson import ObjectId
import bson, json, datetime

def documentHandler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    elif isinstance(x, bson.objectid.ObjectId):
        return str(x)
    else:
        raise TypeError(x)

def defDiffElements(collection: str):
    pipeline = [
        {
            '$project': {
                '_id': 1
            }
        }
    ]
    Local = json.loads(json.dumps(list(MongoClient.LocalClient[db.dbDashboard][collection].aggregate(pipeline=pipeline)),default=documentHandler))
    Cloud = json.loads(json.dumps(list(MongoClient.CloudClient[db.dbDashboard][collection].aggregate(pipeline=pipeline)),default=documentHandler))

    idLocal = [i['_id'] for i in Local]
    idCloud = [i['_id'] for i in Cloud]

    for i in idLocal:
        for j in idCloud:
            if i == j:
                idLocal.remove(i)
                idCloud.remove(j)

    for id in idCloud:
        try:
            if MongoClient.CloudClient.is_primary:
                MongoClient.CloudClient[db.dbDashboard][collection].delete_one({'_id': ObjectId(id)})
        except Exception as e:
            eStr = str(e)
            print(eStr) 

    for id in idLocal:
        try:
            document =  MongoClient.LocalClient[db.dbDashboard][collection].find_one({'_id': ObjectId(id)})
            for upTry in range(0,5):
                if MongoClient.CloudClient.is_primary and MongoClient.CloudClient[db.dbDashboard][collection].insert_one(document).acknowledged:
                    break
        except Exception as e:
            eStr = str(e)
            print(eStr)  

    
defDiffElements('Battery_actual')

In [16]:
# Global imports
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, DashboardCollections as col
import GlobalSets.l2c_functions as l2c

# Import librarys
from pymongo import collection, errors
import bson, datetime, json, bson

def documentHandler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    elif isinstance(x, bson.objectid.ObjectId):
        return str(x)
    else:
        raise TypeError(x)


pipelineCol = MongoClient.LocalClient[db.dbDashboard]['pipelines']

def newPipeline(name: str, database: str, collection: str, pipeline: dict):
    document = {
        'name' : name,
        'database': database,
        'collection': collection,
        'pipeline' : pipeline
    }
    try:
        if pipelineCol.insert_one(document=document).acknowledged:
            print('Added')
    except Exception as e:
        eStr = str(e)
        print(eStr)

def executePipelines():
    pipelines = json.loads(json.dumps(list(pipelineCol.find({})),default=documentHandler))
    for pipeline in pipelines:
        MongoClient.LocalClient[pipeline['database']][pipeline['collection']].aggregate(pipeline=pipeline['pipeline'])

executePipelines()
            

ImportError: cannot import name 'DashboardCollections' from 'GlobalSets.Mongo' (/media/andre/98201a4b-9b4b-4da7-aeaa-59369694a00b/GitHub/UGV-backend/DatabaseSync/GlobalSets/Mongo.py)

In [22]:
from runPipelines import newPipeline

pipeline = [
    {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }, {
        '$out': {
            'db': 'CeDRI_UGV_dashboard', 
            'coll': 'Battery_actual'
        }
    }
]

newPipeline("Get actual battery", "CeDRI_UGV_buffer", "Battery_Data", pipeline=pipeline)
    

Added


In [13]:
import pymongo, bson, datetime
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col


# for i in range(0,10):
action = {
    '_id': bson.ObjectId(),
    'graph': 'line',
    'name': 'battery-voltage-time-area',
    'title': 'Battery voltage by time',
    'group': 'Battery',


    'query': {
	"dataSource": "CeDRI",
	"database": "CeDRI_UGV_datalake",
	"collection": "Battery",
	"pipeline": [
		{
			'$project': {
				'dateTime': {
					'$dateTrunc': {
						'date': '$dateTime', 
						'unit': 'minute'
					}
				}, 
				'voltage': 1,
			}
		}, {
			'$densify': {
				'field': 'dateTime', 
				'range': {
					'step': 1, 
					'unit': 'minute', 
					'bounds': 'full'
				}
			}
		}, {
			'$group': {
				'_id': '$dateTime', 
				'voltage': {
					'$avg': '$voltage'
				}
			}
		}, {
			'$sort': {
				'_id': -1
			}
		}, {
			'$limit': 100
		}
	]
},
    
    'config': {
		'xField': '_id',
		'yField': 'voltage',
		'xAxis': {
			'tickCount': 5,
			'type': 'time',
			'mask': 'DD/MMM/YY HH:mm',
			'title': {
				'text': "Time"
			}
		},
		'yAxis':{
			'tickCount': 10,
			'title': {
				'text': "Battery voltage [V]"
			}
		},'tooltip': {
			'formatter': "(data) => { if (data['voltage'] != null){ return { name: 'Voltage', value: data['voltage'].toFixed(1) + ' V' }; } return {}; }",	
		},
	}
}

print(action)
print(MongoClient.RemoteUnitClient['CeDRI_UGV_dashboard']['graphs'].insert_one(action).acknowledged)

# print(action)

{'_id': ObjectId('642dc2568eecef33fe04e93b'), 'graph': 'line', 'name': 'battery-voltage-time-area', 'title': 'Battery voltage by time', 'group': 'Battery', 'query': {'dataSource': 'CeDRI', 'database': 'CeDRI_UGV_datalake', 'collection': 'Battery', 'pipeline': [{'$project': {'dateTime': {'$dateTrunc': {'date': '$dateTime', 'unit': 'minute'}}, 'voltage': 1}}, {'$densify': {'field': 'dateTime', 'range': {'step': 1, 'unit': 'minute', 'bounds': 'full'}}}, {'$group': {'_id': '$dateTime', 'voltage': {'$avg': '$voltage'}}}, {'$sort': {'_id': -1}}, {'$limit': 100}]}, 'config': {'xField': '_id', 'yField': 'voltage', 'xAxis': {'tickCount': 5, 'type': 'time', 'mask': 'DD/MMM/YY HH:mm', 'title': {'text': 'Time'}}, 'yAxis': {'tickCount': 10, 'title': {'text': 'Battery voltage [V]'}}, 'tooltip': {'formatter': "(data) => { if (data['voltage'] != null){ return { name: 'Voltage', value: data['voltage'].toFixed(1) + ' V' }; } return {}; }"}}}
True


In [7]:
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
print(MongoClient.RemoteUnitClient['CeDRI_UGV_dashboard']['config'].find_one_and_update(upsert=True, filter={'user': "default"}, update= { '$set': { "Branch" : 'ECE'} }, ))

{'_id': ObjectId('6400d26e1f6457c6e47ee729'), 'user': 'default', 'password': '0000', 'dashboardLayout': {'00': {'graph': 'battery-percent-time-area', 'type': 'simple'}, '01': {'graph': 'battery-voltage-time-area', 'type': 'simple'}, '02': {'graph': 'cpu-cpu-time-area', 'type': 'simple'}, '10': {'graph': 'connection-rtt-time-area', 'type': 'simple'}, '11': {'graph': 'cpu-user-sun-brush', 'type': 'simple'}, '12': {'graph': 'memory-user-sun-brush', 'type': 'simple'}, '20': {'graph': 'log-actual-status-log', 'type': 'large'}, '21': {'graph': 'map-heat-RTT-map', 'type': 'big'}}, 'Branch': 'ECE'}


In [51]:
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
import json


def my_handler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    elif isinstance(x, bson.objectid.ObjectId):
        return str(x)
    elif isinstance(x, float) and math.isnan(x):
        return None
    else:
        print(x)
        print('_______________________________________________________________________________________________________')
        raise TypeError(x)

def nan2None(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    elif isinstance(obj, bson.objectid.ObjectId):
        return str(obj)
    elif isinstance(obj, dict):
        return {k:nan2None(v) for k,v in obj.items()}
    elif isinstance(obj, list):
        return [nan2None(v) for v in obj]
    elif isinstance(obj, float) and math.isnan(obj):
        return None
    return obj
class NanConverter(json.JSONEncoder):
    def default(self, obj):
        my_handler(obj)
        pass
    def encode(self, obj, *args, **kwargs):
        obj = nan2None(obj)
        return super().encode(obj, *args, **kwargs)
    def iterencode(self, obj, *args, **kwargs):
        obj = nan2None(obj)
        return super().iterencode(obj, *args, **kwargs)

# a =  .find()
a = MongoClient.RemoteUnitClient['CeDRI_dashboard']['robots'].find_one(filter={'robot': 'bigCeDRI'}, projection={'robot': 1})
print(a)

{'_id': ObjectId('6450d1209e158f8fb6e3ac11'), 'robot': 'bigCeDRI'}
